In [1]:
from alpaca.trading.client import TradingClient, GetAssetsRequest
import os
import config as cg
from alpaca.trading.enums import OrderSide, OrderType, TimeInForce
from alpaca.trading.requests import MarketOrderRequest, LimitOrderRequest, StopOrderRequest, StopLimitOrderRequest
from alpaca.trading.stream import TradingStream
from alpaca.data.requests import CryptoLatestQuoteRequest
from alpaca.data.historical import CryptoHistoricalDataClient
import numpy as np
import pandas as pd

client = TradingClient(cg.api_key, cg.secret_key, paper=True)

account = dict(client.get_account())
df = pd.DataFrame(account, index=[0])   
df = df[['buying_power', 'status', 'cash']]
df

,buying_power,status,cash
0,2.74,AccountStatus.ACTIVE,1.37


In [2]:
data_client = CryptoHistoricalDataClient()

request_params = CryptoLatestQuoteRequest(symbol_or_symbols="BTC/USD")

latest_quote = data_client.get_crypto_latest_quote(request_params)
latest_quote

{'BTC/USD': {   'ask_exchange': None,
     'ask_price': 64201.49,
     'ask_size': 0.2787,
     'bid_exchange': None,
     'bid_price': 64144.1,
     'bid_size': 0.2762,
     'conditions': None,
     'symbol': 'BTC/USD',
     'tape': None,
     'timestamp': datetime.datetime(2024, 8, 25, 4, 48, 2, 384380, tzinfo=TzInfo(UTC))}}

In [150]:
df_price = pd.concat({k: pd.DataFrame(v) for k, v in latest_quote.items()}, axis=0)
df_price.drop(columns=[0], inplace=True)
df_price.rename(index={0: 'symbol', 1:'time', 2:'ask_price', 3:'ask_size', 4:'bid_exchange', 5:'bid_price', 6:'bid_size' }, inplace=True)
df_price = df_price.T
df_price.columns = df_price.columns.droplevel(0)
df_price['ask_price']

1    60542.0
Name: ask_price, dtype: object

In [ ]:
order_details = MarketOrderRequest(symbol='BTCUSD', qty=0.01, side=OrderSide.BUY, time_in_force=TimeInForce.GTC)

order = client.submit_order(order_details)